Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = ""
COLLABORATORS = "Aitor Espinós & Guillem Perales"

---

# Python assignment

The python assignment you can do either on your own or with one other student (i.e. max group size is 2 students).

The first cell of your notebook, should contain a table with the names and SNRs and ANRs of the group members, like so

|Name|SNR|ANR|
|----|---|----|
|Aitor Espinós|2069780|u202365|
|Guillem Perales|2066999|u725893|



See [the webpage](https://janboone.github.io/applied-economics/#org065a005) for details of what we expect to see in this assignment.



# Research question (0.5 points)

Formulate the research question: what question do you want to answer in this assignment?

1. Do community-based societies lead to a better allocation of resources, so, to higher population?
2. Do societies tend to cluster into communities?
3. Does the individual capacity/ability influence the incentive to cluster into communities?
4. Is there a steady state? Is it unique for each endowment?

# Motivation (0.5 points)

Motivate why this question is interesting.

YOUR ANSWER HERE

# Method (0.5 points)

If you do a theory notebook (without external data; but perhaps with data that you simulate yourself), explain the methods that you use for the theory:

* is there an optimization problem underlying the notebook (like a firm maximizing profits)?
* do you calculate an equilibrium?
* what model do you use?

# [optional] Data [0.5 points]

**These 0.5 points are instead of the Method points above**

If you use data for your assignment, the Method part above does not apply. Instead,

* explain where your data come from (give a link or the code to download the data if you use an API like [wbdata](https://wbdata.readthedocs.io/en/stable/));
* explain the data cleaning and data normalization steps that you use;
* make sure that the data is in your github repository as well (or explain why this is not possible) so that we can replicate your analysis;
* what statistical methods do you use with the data.


1. The simulation starts with 'n' number of individuals. Each individual 'i' is part of a community 'c', and has a level of ability to find food 'a'.
2. Each period 'p', each individual finds 'f' pieces of food (where 'f' is a list (of random length depending on 'a') of different (but similar) pieces of food (which can go from 0 to 'y')).
3. If a specific piece of food is included inside the 'f' of 2 or more individuals:
    1. Individuals of different communities will fight each other resulting in a probability 'd' of dying for each individual fought.
    2. The piece of food will be randomly allocated to one of the individuals that survived.
4. If in a community there is an individual with more than 1 piece of food and an individual with less than 1 piece of food, the one with more gives 1 piece to the one with less.
5. Individuals with 0 pieces of food die.
6. Each individual has a probability 'l' of leaving its community.
7. Individuals that leave have a probability 'j' of joining another random community, or, otherwise, creating its own community.
7. Each individual has a probability 'r' of reproducing, creating a new individual with the same characteristics of its progenitor.

# Preview of the answers (0.5 points)

Summarize the results that you find and the answer to your research question.

YOUR ANSWER HERE

# Main assumptions (0.5 points)

What are the main assumptions that you need to answer the question? If you do a theory notebook, what are the main assumptions of your model? If you use data, what assumptions are needed by your estimation method.

1. No technological growth.
2. Closed frontiers.
3. Individuals have no information on other individuals.
4. Leaving or joining a community is exogenous.

# Python code (6.5 points)


Give the python code in code cells and use markdown cells to explain why you code things in this way and what the outcomes are of the code cells.

Note that the explanation of your code is at least as important as the python code itself.

Create as many python and markdown cells as you need to explain things well.

First, we import all the libraries we will need for the simulation and its further analysis:

In [2]:
import random as rd
import numpy as np
import pandas as pd
import matplotlib.pyplot as mp
import sqlite3 as sq

Then, we will start by setting the number of periods that the simulation will have:

In [3]:
p=40

We continue by setting the initial number of individuals that will exist in the simulation:

In [4]:
n=4

We set the minimum and the maximum level of ability to find food that an individual can have in the simulation:

In [5]:
min_a=1
max_a=3

The maximum amount of communities that the simulation will start with:

In [6]:
c=2

And the minimum and maximum amount of food (or savings) with which each individual can start with:

In [7]:
min_s=0
max_s=0

For the simulation to kick off, we will first need to assign initial values for the variables: 'ability', 'community', and 'savings'.

The variable 'ability' will be a list in which the index of the list will represent each individual (sorted by age), and the value will indicate the ability that the individual has to find food:

In [8]:
ability=[]

The initial values of the variable 'ability' will be assigned independently for each starting individual, taking a random value from 'min_a' to 'max_a' each:

In [9]:
for i in range(n):
    ability.append(rd.randint(min_a,max_a))
print(ability)

[3, 1, 2, 2]


The variable 'community' will be a 2-dimensional array in which the row index will represent each period, the column index will represent each individual (sorted by age), and the value will indicate in which community does the individual belong at the start of the indicated period:

In [10]:
community=[[]]

The initial values of the variable 'community' will be assigned independently for each starting individual, taking a random value from 1 to 'c' each:

In [11]:
for i in range(n):
    community[0].append(rd.randint(1,c))
print(community)

[[2, 1, 1, 2]]


_It is important to take into account that we will use the variable 'community' aswell to know, in each period, for each individual, whether the individual is alive or not: a value of 0 in the variable 'community' will indicate that the individual in the indicated period is not alive (eiter dead or not yet born)._

The variable 'savings' will be a 2-dimensional array in which the row index will represent each period, the column index will represent each individual (sorted by age), and the value will indicate the amount of stored food that the individual has at the start of the indicated period:

In [12]:
savings=[[]]

The initial values of the variable 'savings' will be assigned independently for each individual, taking a random value from 'min_s to 'max_s' each:

In [13]:
for i in range(n):
    savings[0].append(rd.randint(min_s,max_s))
print(savings)

[[0, 0, 0, 0]]


We will create a last 2-dimensional array called 'food_found', in which the row index will represent each period, the column index will represent each individual (sorted by age), and the value will indicate the amount of food that the individual has found during the indicated period _(we will not need to assign initial values for this variable)_:

In [14]:
food_found=[]

And we will finally need to set the maximum amount of food that can be found in total by all the individuals each period:

In [15]:
max_f=20

Now, we will define as functions the different processes (or steps) that will take place each period in the simulation:

In the first place, alive individuals will find food depending on their ability level. They will find a random amount of food that will go from 0 to their 'ability'. Not alive individuals will directly find 0 food.

Then, if the sum of food found by all individuals in the period happened to exceed the maximum amount of food 'max_f', the difference between the sum and 'max_f' would be randomly taken away, unit by unit, from all individuals whose 'food_found' in period 1 is higher than 0:

In [16]:
def find_food(food_found,ability,community,t,max_f):
    food_found.append([])
    for i in range(len(ability)):
        if community[t][i]!=0:
            food_found[t].append(rd.randint(0,ability[i]))
        else:
            food_found[t].append(0)
    while max_f<sum(food_found[t]):
        chosen=rd.randint(0,len(ability)-1)
        if 0<food_found[t][chosen]:
            food_found[t][chosen]-=1

In the second place, we will sum the savings that each individual started the period with, plus the amount of food found by the individual during the same period; and we will store the resulting values as the savings with which the individual will start the next period.

Then, alive individuals with 0 pieces of food (that would die at the end of the period because they would be unable to eat), are given 1 piece of food by someone from the same community, given that this 'someone' has at least 2 pieces of food. Younger individuals would be prioritized in case of doubt:

In [17]:
def store_and_share(savings,ability,t,food_found,community):
    savings.append([])
    for i in range(len(ability)):
        savings[t+1].append(savings[t][i]+food_found[t][i])
    for i in reversed(range(len(ability))):
        if community[t][i]!=0 and savings[t+1][i]<1:
            for j in range(len(ability)):
                if community[t][j]==community[t][i] and 1<savings[t+1][j]:
                    savings[t+1][j]-=1
                    savings[t+1][i]+=1
                    break

In the third place, almost at the end of the period, individuals with at least 1 piece of food would consume 1 piece to survive, and individuals that don't have food would die:

In [18]:
def survive(community,ability,savings,t):
    community.append([])
    for i in range(len(ability)):
        if 0<savings[t+1][i]:
            savings[t+1][i]-=1
            community[t+1].append(community[t][i])
        else:
            community[t+1].append(0)

In the fourth place, there is a chance 'l' that alive individuals leave their community, and if they do so, they will join any other community or create a new one, all options with equal probability:

In [19]:
l=0.02
def change_community(ability,community,t,l):
    for i in range(len(ability)):
        if community[t+1][i]!=0 and rd.uniform(0,1)<l:
            while community[t+1][i]==community[t][i]:
                community[t+1][i]=rd.randint(1,max(community[t])+1)

And finally, at the end of the period, each alive individual will have a chance 'r' of reproducing. The newborn individual will share the same characteritics as his/her progenitor, but with 'savings' equal to 0:

In [20]:
r=0.2
def reproduce(ability,community,t,r,savings,food_found):
    for i in range(len(ability)):
        if community[t+1][i]!=0 and rd.uniform(0,1)<r:
            for u in range(t+1):
                community[u].append(0)
                savings[u].append(0)
                food_found[u].append(0)
            ability.append(ability[i])
            community[t+1].append(community[t+1][i])
            savings[t+1].append(0)

In [21]:
def simulate(p,food_found,ability,community,max_f,savings,l,r):
    for t in range(p):
        find_food(food_found,ability,community,t,max_f)
        store_and_share(savings,ability,t,food_found,community)
        survive(community,ability,savings,t)
        change_community(ability,community,t,l)
        reproduce(ability,community,t,r,savings,food_found)

simulate(p,food_found,ability,community,max_f,savings,l,r)
print('ability: '+str(ability))
print('community first period: '+str(community[0]))
print('community last period: '+str(community[p]))
print('savings last period: '+str(savings[p]))

ability: [3, 1, 2, 2, 1, 2, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 2, 3, 3, 3, 3, 3, 2, 3, 3, 3, 2, 3, 3, 2, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
community first period: [2, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
community last period: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

# Sensitivity analysis (0.5 points)

If you do a theory notebook, program different values for the parameters, use different functional forms and solve your model again for these new values. Show that the outcomes are robust to such changes.

If you are using data, how sensitive are your results to outliers in the data, different choices when cleaning the data, different functional forms for the relations that you estimate.


In [23]:
# YOUR CODE HERE

# Discussion and conclusion (0.5 points)

What did you find in the analysis above; what is the answer to the question you started out with.

What are weaknesses of your approach that can be improved upon in future research (e.g. in your thesis).


We are supposing a closed economy with no technology improvement.